# This notebook is for further cleaning of the glider data for plotting

In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import numpy as np
import os
import matplotlib.dates as mdates
import cmocean as cm
import glob
import os
import waypoint_distance as wd
%matplotlib widget

# Plotting function

In [ ]:
def plot_mission(ds, lat_bounds=None, lon_bounds=None):
    """
    Plot a single glider path over bathymetry for the Calvert Line.

    Parameters:
    - file_path: str, optional, used to extract mission name
    - lon_bounds, lat_bounds: optional bounds for plotting
    """
    
    topo_file=os.path.expanduser('~/Desktop/british_columbia_3_msl_2013.nc')

    # Extract lon/lat
    lons = ds['longitude'].values
    lats = ds['latitude'].values
    time_vals = ds['time'].values
    time_nums = mdates.date2num(time_vals)

    # Auto bounds if not provided
    if lon_bounds is None:
        lon_bounds = [lons.min() - 0.5, lons.max() + 0.5]
    if lat_bounds is None:
        lat_bounds = [lats.min() - 0.5, lats.max() + 0.5]

    # Load topo data
    topo = xr.open_dataset(topo_file)
    topo = topo.sel(
        lon=slice(lon_bounds[0], lon_bounds[1]),
        lat=slice(lat_bounds[0], lat_bounds[1]))

    # Set up plot
    fig, ax = plt.subplots(figsize=(12, 9), subplot_kw={'projection': ccrs.PlateCarree()})
    ax.set_extent(lon_bounds + lat_bounds, crs=ccrs.PlateCarree())
    gl = ax.gridlines(draw_labels=True, linestyle='--', alpha=0)
    gl.top_labels = False
    gl.right_labels = False

    # Plot bathymetry
    topo_var = -topo['Band1']
    levels = np.linspace(0, 410, 51)
    contourf = ax.contourf(topo['lon'], topo['lat'], topo_var,
                           levels=levels, cmap=cm.cm.deep, extend='both')
    fig.colorbar(contourf, ax=ax, label='Depth (m)')

    # Add 0 m elevation contour in black
    ax.contour(topo['lon'], topo['lat'], topo_var, levels=[0.5], colors='black', linewidths=1)

    # Plot glider track
    sc = ax.scatter(lons, lats, c=time_nums, cmap='seismic',
                    vmin=time_nums.min(), vmax=time_nums.max(),
                    s=5, transform=ccrs.PlateCarree(), label='Filtered Track')
    cbar = fig.colorbar(sc, ax=ax, orientation='vertical', extend='both')
    cbar.set_label('Date')
    cbar.ax.yaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))

    # Title
    if file_pathway:
        mission_name = os.path.basename(file_pathway).split('_')[0]
        ax.set_title(f'Cleaned Glider Mission Track – {mission_name}', fontsize=14)
    else:
        ax.set_title('Cleaned Glider Mission Track', fontsize=14)

    plt.tight_layout()

# Load topo data
topo_file=os.path.expanduser('~/Desktop/british_columbia_3_msl_2013.nc')
topo = xr.open_dataset(topo_file)

# Define along coordinate system, clean data for plotting

In [ ]:
def clean_and_plot_mission(file_pathway, lat_bounds=(51.2, 52), long_bounds=(-128.9, -127.75)):
    """
    Clean and plot a glider mission by separating outbound and return transects,
    removing redundant points, and plotting the results.
    """
    ds = xr.open_dataset(file_pathway)

    # Waypoints
    waypoint_lon = np.array([-127.950, -128.115, -128.243, -128.514, -128.646, -128.798])
    waypoint_lat = np.array([51.757, 51.705, 51.715, 51.450, 51.4165, 51.408])
    central_lat = 51.715

    # Distance projection
    alongx, acrossx, _ = wd.get_simple_distance(
        shiplon=ds['longitude'].values,
        shiplat=ds['latitude'].values,
        wplon=waypoint_lon,
        wplat=waypoint_lat,
        central_lat=central_lat)

    # Add along and across to ds
    ds = ds.assign(along=('time', alongx))
    ds = ds.assign(across=('time', acrossx))

    # Separate ds into out and return transects
    gradient_along = np.gradient(ds['along'])
    zero_indices = np.where(gradient_along == 0)[0]
    # Indices to figure out how the out and return should be separated
    # Requires gradient_along to have zeros in the middle, which are created from the projection function and it's bounds
    start = zero_indices[0]
    end = zero_indices[-1] + 1

    # Creating the masks
    out_mask = np.zeros_like(gradient_along, dtype=bool)
    return_mask = np.zeros_like(gradient_along, dtype=bool)
    out_mask[:start] = gradient_along[:start] > 0
    return_mask[end:] = gradient_along[end:] < 0

    # Defining new ds, one for each part of the trip
    ds_out = ds.sel(time=out_mask)
    ds_return = ds.sel(time=return_mask)

    # Iteratively remove points from glider loops (outbound)
    prev_len = -1
    while prev_len != len(ds_out['time']):
        prev_len = len(ds_out['time'])
        grad = np.gradient(ds_out['along'])
        keep_mask = grad > 0
        ds_out = ds_out.sel(time=keep_mask)

    # Iteratively remove points from glider loops (returnbound)
    prev_len = -1
    while prev_len != len(ds_return['time']):
        prev_len = len(ds_return['time'])
        grad = np.gradient(ds_return['along'])
        keep_mask = grad < 0
        ds_return = ds_return.sel(time=keep_mask)

    # Combine back into ds, plot. Data is now clean, not yet horizontally gridded
    ds = xr.concat([ds_out, ds_return], dim='time')
    plot_mission(ds, lat_bounds = lat_bounds, lon_bounds = long_bounds)
    # plot_mission(ds_out, file_path = '/Users/martinwilliamson/Desktop/dfo-bb046-20210324_grid_delayed.nc',  lat_bounds = lat_bounds, long_bounds = long_bounds)
    # plot_mission(ds_return, file_path = '/Users/martinwilliamson/Desktop/dfo-bb046-20210324_grid_delayed.nc',  lat_bounds = lat_bounds, long_bounds = long_bounds)
    return ds, ds_out, ds_return

# Example usage
if __name__ == "__main__":
    file_pathway = '/Users/martinwilliamson/Desktop/dfo-bb046-20210324_grid_delayed.nc'
    # lat_bounds = (51.2, 52)
    # lon_bounds = (-128.9, -127.75)
    ds, ds_out, ds_return = clean_and_plot_mission(file_pathway)

# Check time gradients, see that the time difference is reasonable over the ds

In [ ]:
if __name__ == "__main__":
    # Compute time gradients as timedelta64[ns]
    time_grad_out = np.gradient(ds_out['time'].values.astype('datetime64[ns]'))
    time_grad_return = np.gradient(ds_return['time'].values.astype('datetime64[ns]'))

    # Print raw gradients in seconds
    print(f'Time differences between profiles for the OUT trip (seconds): {time_grad_out.astype("timedelta64[s]")}')
    print(f'Time differences between profiles for the RETURN trip (seconds): {time_grad_return.astype("timedelta64[s]")}')

    # Compute and convert max differences to hours
    max_time_diff_out = np.max(time_grad_out).astype('timedelta64[ns]').astype(np.int64)
    max_time_diff_return = np.max(time_grad_return).astype('timedelta64[ns]').astype(np.int64)

    print(f'Maximum time difference in the OUT trip: {max_time_diff_out / 3.6e12:.2f} hours')
    print(f'Maximum time difference in the RETURN trip: {max_time_diff_return / 3.6e12:.2f} hours')

# Interpolating

In [ ]:
def interpolate(ds):
    """
    Interpolate the dataset to a regular grid along the 'along' dimension.
    """
    # Drop duplicates in the 'along' dimension
    _, index_unique = np.unique(ds['along'], return_index=True)
    ds = ds.isel(time=index_unique)
    
    # Ensure 'along' is a coordinate
    if 'along' not in ds.coords:
        raise ValueError("'along' must be a coordinate in the dataset.")

    # Create a new grid for interpolation
    along_grid = np.arange(ds['along'].min(), ds['along'].max(), 200)  # resolution of 200m

    # Save the original 'time' variable
    original_time = ds['time'].values

    # Swap dimensions and interpolate
    ds_swapped = ds.swap_dims({'time': 'along'})
    ds_interpolated = ds_swapped.interp(along=along_grid)

    # Reassign 'time' to the interpolated dataset
    # Use linear interpolation to estimate time values for the new 'along' grid
    ds_interpolated['time'] = ('along', np.interp(along_grid, ds['along'].values, original_time.astype(float)))

    return ds_interpolated

# Plot Sections #

In [ ]:
def plot_section(ds_out, ds_return):
    """
    Plot temperature and potential density sections for both outbound and return transects for out and return transects.
    """
    
    ds_out = ds_out.set_coords('along')
    ds_return = ds_return.set_coords('along')

    # Interpolate the outbound and return datasets
    ds_out = interpolate(ds_out)
    ds_return = interpolate(ds_return)

    for ds, transect_type in zip([ds_out, ds_return], ["Outbound", "Return"]):
        # Interpolate bathymetry at lat/lon positions
        interp_bathy = topo['Band1'].interp(
            lon=xr.DataArray(ds['longitude'].values, dims='time'),
            lat=xr.DataArray(ds['latitude'].values, dims='time'))
        bottom_depths = -interp_bathy
        
        # Extract variables
        along = ds['along'].values
        time = ds['time']
        depth = ds['depth'].values
        temperature = ds['temperature'].values
        pdens = ds['potential_density'].values - 1000  # Sigma-theta

        # ===== Plot Along-Transect Section =====
        fig, ax = plt.subplots(figsize=(12, 6))
        cf = ax.pcolormesh(along, depth, temperature, shading='nearest', cmap=cm.cm.thermal)
        ax.plot(along, bottom_depths, color='black', linewidth=2, label='Bathymetry')

        cf_iso = ax.contour(along, depth, pdens, levels=np.linspace(24, 27, 16),
                            colors='grey', linewidths=0.5, linestyles='-')
        ax.clabel(cf_iso, fmt='%1.2f', fontsize=10)

        ax.set_xlabel('Along-Transect Distance (m)')
        ax.set_ylabel('Depth (m)')
        ax.invert_yaxis()
        ax.set_title(f'Temperature Section ({ds.attrs.get("mission", "Mission")})')
        cf.set_clim(vmin=ds['temperature'].min(), vmax=ds['temperature'].max())
        plt.colorbar(cf, ax=ax, label='Temperature (°C)')
        ax.legend(loc='lower right')
        plt.tight_layout()
        ax.set_ylim(410, 0)
        ax.set_xlim(0, 77000)

        # ===== Plot Time Section =====
        fig, ax = plt.subplots(figsize=(12, 6))
        cf = ax.pcolormesh(time, depth, temperature, shading='nearest', cmap=cm.cm.thermal)
        ax.plot(time, bottom_depths, color='black', linewidth=2, label='Bathymetry')

        cf_iso = ax.contour(time, depth, pdens, levels=np.linspace(24, 27, 16),
                            colors='grey', linewidths=0.5, linestyles='-')
        ax.clabel(cf_iso, fmt='%1.2f', fontsize=10)

        ax.set_xlabel('Time')
        ax.set_ylabel('Depth (m)')
        ax.invert_yaxis()
        ax.set_title(f'Temperature Section (Time) ({ds.attrs.get("mission", "Mission")})')
        plt.colorbar(cf, ax=ax, label='Temperature (°C)')
        ax.legend(loc='lower right')
        plt.tight_layout()
        ax.set_ylim(410, 0)
        # Invert x-axis for return transect in the time plot
        if transect_type == "Return":
            ax.invert_xaxis()

In [ ]:
def process_glider_mission(file_pathway, lat_bounds=(51.2, 52), lon_bounds=(-128.9, -127.75)):
    """
    Clean, plot on a map, and plot sections for both outbound and return glider trips.
    Args:
        file_pathway (str): Path to the glider netCDF file.
        lat_bounds (tuple): Latitude bounds for plotting.
        lon_bounds (tuple): Longitude bounds for plotting.
    """
    # Clean and split the mission
    ds, ds_out, ds_return = clean_and_plot_mission(file_pathway, lat_bounds=lat_bounds, long_bounds=lon_bounds)
    # Plot sections for outbound and return
    plot_section(ds_out, ds_return)

# Example usage:
process_glider_mission('/Users/martinwilliamson/Desktop/dfo-bb046-20210324_grid_delayed.nc')

In [ ]:
ds